Autor: so jae woo    
Data src :  qesg news / 28 cat / 본문 포함.    
Summary : KLUE기반의 버트 사정 임베딩 모델 사용.   
- 원본경로 unction\220727_성능개선작업


In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import transformers
transformers.__version__ # 
from sklearn.model_selection import StratifiedKFold

from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import TextClassificationPipeline

from lib_offline import prep_mecab_noun_offline
from lib2 import text_preprocessing
import pickle

# 데이터 준비 

## QESG 데이터 준비 

In [13]:
df_qesg = pickle.load(open("./data_all_na_news.pkl",'rb'))
df_qesg = df_qesg.dropna( subset = ['contents','label_category'] , axis = 0 ) 

### 데이터 클리닝

In [14]:
%%time
df_qesg['contents_clean'] = df_qesg['contents'].apply(lambda x : text_preprocessing.manual_remove(text_preprocessing.cleaning_korean(x)))
df_qesg = df_qesg[['id', 'keyword', 'title', 'link', 'contents', 'label_category','label_sentiment', 'contents_clean']]

#컬럼명 바꾸기
df_qesg.columns = ['id', 'keyword', 'title', 'link', 'contents', 'y','label_sentiment', 'contents_clean']
df_qesg = df_qesg.reset_index(drop=True)

CPU times: total: 30.7 s
Wall time: 28.5 s


### 짧은 본문 제거

In [15]:
def check_contents_clean(row):
    c = row.contents
    cc = row.contents_clean
    
    if len(c) == 0:
        return False
    
    if len(cc)/len(c)> 0.7:
        return True
    else:
        return False

In [19]:
df_qesg = df_qesg.loc[df_qesg.y != '제외']
bol_list = [ check_contents_clean(row) for i , row in df_qesg.iterrows() ]
df_qesg['size'] = df_qesg[bol_list].contents_clean.apply(lambda x : len(x))
df_qesg = df_qesg.loc[df_qesg['size'] > 40]

### 카테고리 28->35개로 변경

In [20]:
df35 = pd.read_csv('./카테고리_28to35.csv').dropna()
dict_2835 = dict(zip(list(df35.before) , list(df35.after)))

res = []
for item in df_qesg.y:
    try:
        res.append(dict_2835[item])
    except:
        res.append('**'+item)
        
df_qesg['y2'] = res

In [22]:
#최소 필요 컬럼만 가져오기
df_qesg_small = df_qesg[[ 'link', 'contents','contents_clean','y', 'y2']]

### 인적자원관리 쪽 실분류 데이터 가져오기 

In [24]:
df_new = pd.read_excel('./인적자원관리_qesg데이터_기업명필터_김신영 재분류.xlsx')
df_new.columns = ['순번', 'pidx', 'title', 'date', 'link', 'esg_predict', 'label_category','표준기업명', 'y', 'allow']
df_new = df_new[['link',  'y']]
df_new.columns = ['link',  'y2']


# qesg에서 인적자원관리 빼버리기 
df_qesg_noin = df_qesg_small.loc[df_qesg_small.y != '인적자원관리']

# qesg에서 인적자원 관리만 가져오기 
df_qesg_yesin = df_qesg_small.loc[df_qesg_small.y == '인적자원관리']

# df_new에 contents_clean 넣기 
df_new_con = df_new.merge(df_qesg_yesin.drop('y2',axis = 1), on = 'link', how = 'left' )
df_new_con = df_new_con.dropna()
df_new_con = df_new_con[['link', 'contents', 'contents_clean', 'y','y2']]

In [25]:
#관련없음 빼기
df_new_con = df_new_con.loc[df_new_con.y2 != '관련 없음']

# 두개를 합치기 
df_35 = pd.concat([df_qesg_noin,df_new_con], ignore_index=True)

# 제거 카테고리
df_35 = df_35.loc[df_35.y2 != '**소유구조']
df_35 = df_35.loc[df_35.y2 != '**사회공헌']
df_35 = df_35.loc[df_35.y2 != '**이익배분']
df_35 = df_35.loc[df_35.y2 != '**ESG 거버넌스']

df_35 = df_35.reset_index()

### 평가판 데이터 및 본문 가져오기 

In [28]:
''' 데이터 로드 '''
df_res = pd.read_excel('./평가판/220715_평가판예측결과_35분류명칭수정_인적자원관리_김신영 작업완료.xlsx', index_col=0)
df_res_con = pd.read_feather('./평가판/평가판본문.feather')


''' 평가판 레이블 가져오기'''
#df_res = df_res[['기업명', '상장코드', 'link', 'title','평가문항키워드', '(김신영) 1순위 카테고리']]
df_res = df_res[[ 'link', '평가문항키워드', '(김신영) 1순위 카테고리']]
df_res.columns = [ 'link', 'y','y2']
df_res = df_res.loc[df_res.y2 != 'X']
df_res = df_res.loc[df_res.y2 != 'x']
df_res = df_res.drop_duplicates(subset = ['link'])

# 평가판 본문 가져오기 : link - contents - contetns_clean
df_res_con = pd.read_feather('./평가판/평가판본문.feather')
df_nona = df_res_con.dropna(subset = ['contents','title'])
df_nona = df_nona.drop_duplicates(subset = ['link'])

# contents_noun 만들기
df_n = prep_mecab_noun_offline.content2noun_mecab(df_nona['contents'])
df_res_con = pd.concat([df_nona.reset_index(drop=True) , df_n.reset_index(drop=True)] , axis= 1)
df_res_con = df_res_con[['link', 'contents','contents_nouns']]

# contents clean 만들기
ccleans = []
for c in df_res_con.contents:
    c = text_preprocessing.cleaning_korean(c)
    c = text_preprocessing.manual_remove(c)
    ccleans.append(c)
df_res_con['contents_clean'] = ccleans


#합치기
dfp = df_res.merge(df_res_con, on = 'link' , how = 'inner')
dfp = dfp[['link', 'contents', 'contents_clean' ,'contents_nouns', 'y', 'y2']]

  0%|          | 0/2769 [00:00<?, ?it/s]

### 일부 클래스 채우기

In [29]:

minor1 = ['토양오염' , '수질오염' , '대기오염' , '환경영향평가 미흡 ', '생물다양성 저해', '유해화학물질 배출' , '폐기물 배출' ]
dfs = []
for col in minor1:
    df_temp = df_35.loc[df_35.y2 == col]
    if len(df_temp) > 200 : 
        df_temp = df_temp.sample(n = 200, random_state = 2022)
    dfs.append(df_temp)
    
df_minor = pd.concat(dfs, ignore_index=True)
df_minor = df_minor.drop('index',axis = 1)

''' 본문 형태소 분석 한 것 만들기 '''
df_nona = df_minor.drop_duplicates(subset = ['link'])
df_n = prep_mecab_noun_offline.content2noun_mecab(df_nona['contents'])
df_res_con = pd.concat([df_minor.reset_index(drop=True) , df_n.reset_index(drop=True)] , axis= 1)
df_res_con = df_res_con[['link', 'contents_nouns']]

#합치기
df_minor = df_minor.merge(df_res_con, on = 'link' , how = 'inner')
df_minor = df_minor[['link', 'contents', 'contents_clean','contents_nouns', 'y', 'y2']]

  0%|          | 0/610 [00:00<?, ?it/s]

In [30]:
df_minor_fill = pd.concat([dfp, df_minor] ,ignore_index=True).reset_index()
df_minor_fill = df_minor_fill.reset_index()

### 형태소분석 추가

In [31]:
''' qesg데이터 형태소분석  '''
df_nona = df_35.drop_duplicates(subset = ['link'])
df_n = prep_mecab_noun_offline.content2noun_mecab(df_nona['contents'])
df_res_con = pd.concat([df_nona.reset_index(drop=True) , df_n.reset_index(drop=True)] , axis= 1)
df_res_con = df_res_con[['link', 'contents_nouns']]

#합치기
df_35_noun = df_35.merge(df_res_con, on = 'link' , how = 'inner')
df_35_noun = df_35_noun[['link', 'contents','contents_clean', 'contents_nouns', 'y', 'y2']]
df_35_noun = df_35_noun.reset_index(drop = True)

  0%|          | 0/24364 [00:00<?, ?it/s]

In [33]:
df_minor_fill.to_parquet('./df_minor_fill.pq')
df_35_noun.to_parquet('df_35_noun.pq')

In [35]:
# data load
df_minor_fill = pd.read_parquet('./df_minor_fill.pq')
df_35_noun = pd.read_parquet('df_35_noun.pq')